## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 25)                1275      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 10751 (42.00 KB)
Trainable params: 10751 (42.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#define the checkpoint path and filenames
checkpoint_path = "checkpoints/weights.{epoch:02d}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

#create the directory if it doesn't exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

#create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5)

In [18]:
# Implement early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_los', patience=10)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, callbacks=[cp_callback, early_stopping])

Epoch 1/100
  1/686 [..............................] - ETA: 4s - loss: 0.4032 - accuracy: 0.8125
Epoch 1: saving model to checkpoints/weights.01.weights.h5

Epoch 1: saving model to checkpoints/weights.01.weights.h5
 10/686 [..............................] - ETA: 4s - loss: 0.5105 - accuracy: 0.7531
Epoch 1: saving model to checkpoints/weights.01.weights.h5

Epoch 1: saving model to checkpoints/weights.01.weights.h5
 20/686 [..............................] - ETA: 4s - loss: 0.4987 - accuracy: 0.7625
Epoch 1: saving model to checkpoints/weights.01.weights.h5

Epoch 1: saving model to checkpoints/weights.01.weights.h5
 30/686 [>.............................] - ETA: 4s - loss: 0.5117 - accuracy: 0.7552
Epoch 1: saving model to checkpoints/weights.01.weights.h5

Epoch 1: saving model to checkpoints/weights.01.weights.h5
 40/686 [>.............................] - ETA: 4s - loss: 0.5098 - accuracy: 0.7539
Epoch 1: saving model to checkpoints/weights.01.weights.h5

Epoch 1: saving model to ch

686/686 [==============================] - 4s 6ms/step - loss: 0.5306 - accuracy: 0.7398 - val_loss: 0.5910 - val_accuracy: 0.7394
Epoch 2/100
  1/686 [..............................] - ETA: 2s - loss: 0.5476 - accuracy: 0.7812
Epoch 2: saving model to checkpoints/weights.02.weights.h5

Epoch 2: saving model to checkpoints/weights.02.weights.h5
  9/686 [..............................] - ETA: 5s - loss: 0.5162 - accuracy: 0.7847
Epoch 2: saving model to checkpoints/weights.02.weights.h5

Epoch 2: saving model to checkpoints/weights.02.weights.h5
 19/686 [..............................] - ETA: 4s - loss: 0.5069 - accuracy: 0.7780
Epoch 2: saving model to checkpoints/weights.02.weights.h5

Epoch 2: saving model to checkpoints/weights.02.weights.h5
 29/686 [>.............................] - ETA: 4s - loss: 0.5166 - accuracy: 0.7672
Epoch 2: saving model to checkpoints/weights.02.weights.h5

Epoch 2: saving model to checkpoints/weights.02.weights.h5
 39/686 [>.............................] 

686/686 [==============================] - 4s 6ms/step - loss: 0.5293 - accuracy: 0.7406 - val_loss: 0.5928 - val_accuracy: 0.7409
Epoch 3/100
  1/686 [..............................] - ETA: 2s - loss: 0.5153 - accuracy: 0.7188
Epoch 3: saving model to checkpoints/weights.03.weights.h5

Epoch 3: saving model to checkpoints/weights.03.weights.h5
  8/686 [..............................] - ETA: 5s - loss: 0.5346 - accuracy: 0.7461
Epoch 3: saving model to checkpoints/weights.03.weights.h5

Epoch 3: saving model to checkpoints/weights.03.weights.h5
 18/686 [..............................] - ETA: 4s - loss: 0.5413 - accuracy: 0.7344
Epoch 3: saving model to checkpoints/weights.03.weights.h5

Epoch 3: saving model to checkpoints/weights.03.weights.h5
 28/686 [>.............................] - ETA: 4s - loss: 0.5398 - accuracy: 0.7355
Epoch 3: saving model to checkpoints/weights.03.weights.h5

Epoch 3: saving model to checkpoints/weights.03.weights.h5
 38/686 [>.............................] 

686/686 [==============================] - 4s 6ms/step - loss: 0.5334 - accuracy: 0.7401 - val_loss: 0.6223 - val_accuracy: 0.7380
Epoch 4/100
  1/686 [..............................] - ETA: 1s - loss: 0.5560 - accuracy: 0.6875
Epoch 4: saving model to checkpoints/weights.04.weights.h5

Epoch 4: saving model to checkpoints/weights.04.weights.h5
  7/686 [..............................] - ETA: 6s - loss: 0.5771 - accuracy: 0.6875
Epoch 4: saving model to checkpoints/weights.04.weights.h5

Epoch 4: saving model to checkpoints/weights.04.weights.h5
 17/686 [..............................] - ETA: 4s - loss: 0.5514 - accuracy: 0.7298
Epoch 4: saving model to checkpoints/weights.04.weights.h5

Epoch 4: saving model to checkpoints/weights.04.weights.h5
 27/686 [>.............................] - ETA: 4s - loss: 0.5459 - accuracy: 0.7303
Epoch 4: saving model to checkpoints/weights.04.weights.h5

Epoch 4: saving model to checkpoints/weights.04.weights.h5
 37/686 [>.............................] 

686/686 [==============================] - 4s 6ms/step - loss: 0.5301 - accuracy: 0.7391 - val_loss: 0.6044 - val_accuracy: 0.7391
Epoch 5/100

Epoch 5: saving model to checkpoints/weights.05.weights.h5
  1/686 [..............................] - ETA: 18s - loss: 0.4595 - accuracy: 0.8750
Epoch 5: saving model to checkpoints/weights.05.weights.h5

Epoch 5: saving model to checkpoints/weights.05.weights.h5
 11/686 [..............................] - ETA: 4s - loss: 0.5032 - accuracy: 0.7670 
Epoch 5: saving model to checkpoints/weights.05.weights.h5

Epoch 5: saving model to checkpoints/weights.05.weights.h5
 21/686 [..............................] - ETA: 3s - loss: 0.4871 - accuracy: 0.7842
Epoch 5: saving model to checkpoints/weights.05.weights.h5

Epoch 5: saving model to checkpoints/weights.05.weights.h5
 31/686 [>.............................] - ETA: 3s - loss: 0.4958 - accuracy: 0.7742
Epoch 5: saving model to checkpoints/weights.05.weights.h5

Epoch 5: saving model to checkpoints/w

...

686/686 [==============================] - 4s 6ms/step - loss: 0.5289 - accuracy: 0.7410 - val_loss: 0.6092 - val_accuracy: 0.7385
Epoch 97/100
  1/686 [..............................] - ETA: 2s - loss: 0.5367 - accuracy: 0.7188
Epoch 97: saving model to checkpoints/weights.97.weights.h5

Epoch 97: saving model to checkpoints/weights.97.weights.h5
  9/686 [..............................] - ETA: 5s - loss: 0.5249 - accuracy: 0.7292
Epoch 97: saving model to checkpoints/weights.97.weights.h5

Epoch 97: saving model to checkpoints/weights.97.weights.h5
 19/686 [..............................] - ETA: 4s - loss: 0.5271 - accuracy: 0.7385
Epoch 97: saving model to checkpoints/weights.97.weights.h5

Epoch 97: saving model to checkpoints/weights.97.weights.h5
 29/686 [>.............................] - ETA: 4s - loss: 0.5246 - accuracy: 0.7381
Epoch 97: saving model to checkpoints/weights.97.weights.h5

Epoch 97: saving model to checkpoints/weights.97.weights.h5
 39/686 [>......................

686/686 [==============================] - 4s 6ms/step - loss: 0.5295 - accuracy: 0.7404 - val_loss: 0.6137 - val_accuracy: 0.7392
Epoch 98/100
  1/686 [..............................] - ETA: 2s - loss: 0.6020 - accuracy: 0.6562
Epoch 98: saving model to checkpoints/weights.98.weights.h5

Epoch 98: saving model to checkpoints/weights.98.weights.h5
  8/686 [..............................] - ETA: 5s - loss: 0.4830 - accuracy: 0.7773
Epoch 98: saving model to checkpoints/weights.98.weights.h5

Epoch 98: saving model to checkpoints/weights.98.weights.h5
 18/686 [..............................] - ETA: 4s - loss: 0.5133 - accuracy: 0.7500
Epoch 98: saving model to checkpoints/weights.98.weights.h5

Epoch 98: saving model to checkpoints/weights.98.weights.h5
 28/686 [>.............................] - ETA: 4s - loss: 0.5221 - accuracy: 0.7444
Epoch 98: saving model to checkpoints/weights.98.weights.h5

Epoch 98: saving model to checkpoints/weights.98.weights.h5
 38/686 [>......................

686/686 [==============================] - 4s 6ms/step - loss: 0.5294 - accuracy: 0.7409 - val_loss: 0.6041 - val_accuracy: 0.7391
Epoch 99/100
  1/686 [..............................] - ETA: 2s - loss: 0.5560 - accuracy: 0.7500
Epoch 99: saving model to checkpoints/weights.99.weights.h5

Epoch 99: saving model to checkpoints/weights.99.weights.h5
  7/686 [..............................] - ETA: 7s - loss: 0.5925 - accuracy: 0.7321
Epoch 99: saving model to checkpoints/weights.99.weights.h5

Epoch 99: saving model to checkpoints/weights.99.weights.h5
 17/686 [..............................] - ETA: 5s - loss: 0.5571 - accuracy: 0.7445
Epoch 99: saving model to checkpoints/weights.99.weights.h5

Epoch 99: saving model to checkpoints/weights.99.weights.h5
 27/686 [>.............................] - ETA: 4s - loss: 0.5633 - accuracy: 0.7315
Epoch 99: saving model to checkpoints/weights.99.weights.h5

Epoch 99: saving model to checkpoints/weights.99.weights.h5
 37/686 [>......................

686/686 [==============================] - 4s 6ms/step - loss: 0.5300 - accuracy: 0.7400 - val_loss: 0.6038 - val_accuracy: 0.7398
Epoch 100/100

Epoch 100: saving model to checkpoints/weights.100.weights.h5
  1/686 [..............................] - ETA: 18s - loss: 0.5445 - accuracy: 0.7188
Epoch 100: saving model to checkpoints/weights.100.weights.h5

Epoch 100: saving model to checkpoints/weights.100.weights.h5
 11/686 [..............................] - ETA: 3s - loss: 0.5343 - accuracy: 0.7188 
Epoch 100: saving model to checkpoints/weights.100.weights.h5

Epoch 100: saving model to checkpoints/weights.100.weights.h5
 21/686 [..............................] - ETA: 3s - loss: 0.5363 - accuracy: 0.7232
Epoch 100: saving model to checkpoints/weights.100.weights.h5

Epoch 100: saving model to checkpoints/weights.100.weights.h5
 31/686 [>.............................] - ETA: 3s - loss: 0.5388 - accuracy: 0.7218
Epoch 100: saving model to checkpoints/weights.100.weights.h5

Epoch 100: s

686/686 [==============================] - 4s 6ms/step - loss: 0.5295 - accuracy: 0.7406 - val_loss: 0.5928 - val_accuracy: 0.7400


In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.6037 - accuracy: 0.7268 - 294ms/epoch - 1ms/step
Loss: 0.6036771535873413, Accuracy: 0.7268221378326416


In [16]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
